In [11]:
import gradio as gr
from openai import OpenAI
import os
from typing import List, Dict

class HiringAssistant:
    def __init__(self, api_key: str):
        self.client = OpenAI(api_key=api_key)
        
    def generate_technical_questions(self, tech_stack: List[str]) -> List[str]:
        """Generate technical questions based on candidate's tech stack"""
        try:
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "You are a technical interviewer generating targeted assessment questions."},
                    {"role": "user", "content": f"Generate 3-5 technical interview questions for technologies: {', '.join(tech_stack)}. Vary difficulty and assess practical understanding."}
                ]
            )
            questions = response.choices[0].message.content.split('\n')
            return [q.strip() for q in questions if q.strip()]
        except Exception as e:
            return [f"Error generating questions: {str(e)}"]
    
    def process_application(self, 
        name: str, 
        email: str, 
        phone: str, 
        experience: str, 
        position: str, 
        location: str, 
        tech_stack: str
    ) -> str:
        """Process candidate application and generate technical questions"""
        # Validate inputs
        if not all([name, email, phone, experience, position, location, tech_stack]):
            return "Please fill in all fields."
        
        # Process tech stack
        tech_list = [tech.strip() for tech in tech_stack.split(',')]
        
        # Generate technical questions
        technical_questions = self.generate_technical_questions(tech_list)
        
        # Compile application summary
        summary = f"""
        Candidate Application Details:
        Name: {name}
        Email: {email}
        Phone: {phone}
        Years of Experience: {experience}
        Desired Position: {position}
        Location: {location}
        Tech Stack: {', '.join(tech_list)}

        Technical Assessment Questions:
        {chr(10).join([f'{i+1}. {q}' for i, q in enumerate(technical_questions)])}
        """
        
        return summary

def launch_hiring_assistant(api_key: str):
    """Launch Gradio interface for Hiring Assistant"""
    assistant = HiringAssistant(api_key)
    
    interface = gr.Interface(
        fn=assistant.process_application,
        inputs=[
            gr.Textbox(label="Full Name"),
            gr.Textbox(label="Email Address"),
            gr.Textbox(label="Phone Number"),
            gr.Textbox(label="Years of Experience"),
            gr.Textbox(label="Desired Position"),
            gr.Textbox(label="Current Location"),
            gr.Textbox(label="Tech Stack (comma-separated)")
        ],
        outputs=gr.Textbox(label="Application Summary"),
        title="TalentScout Hiring Assistant",
        description="AI-powered candidate screening for tech roles"
    )
    
    return interface

def main():
    # Prompt for API key
    api_key = input("Enter your OpenAI API Key: ")
    
    try:
        app = launch_hiring_assistant(api_key)
        app.launch()
    except Exception as e:
        print(f"Error launching application: {e}")

if __name__ == "__main__":
    main()

Enter your OpenAI API Key:  dfsfsd


Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
